In [23]:
import sys
import os
sys.path.append(os.path.abspath('../src'))
import reach_tools

from datetime import datetime, timedelta
import time
import requests
import pandas as pd

from arcgis.gis import GIS

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
reach = reach_tools.Reach.get_from_aw(3066)

In [25]:
print(reach.reach_search, reach.gauge_id, reach.gauge_observation, reach.gauge_metric, reach.gauge_runnable)
print(f'Flow Min: {reach.gauge_min}')
print(f'Flow Max: {reach.gauge_max}')

Canyon Creek (Lewis River trib.) 2 - Fly Creek to Merwin Reservoir 35665 2470.0 2 False
Flow Min: 1100.0
Flow Max: 1500.0


In [26]:
for rng in reach._reach_json['guagesummary']['ranges']:
    print(float(rng['gauge_min']), float(rng['gauge_max']), rng['range_min'], rng['range_max'])

500.0 700.0 R0 R5
700.0 1100.0 R4 R6
1100.0 1500.0 R5 R9


In [27]:
gis = GIS(username='knu2xs', password='K3mosabe')
gis

GIS @ https://knu2xs.maps.arcgis.com version:7.1

In [33]:
reach_centroid_url = 'https://services3.arcgis.com/p9emohPzcQpdUrEc/arcgis/rest/services/Reach_Centroid/FeatureServer'
reach_centroid_lyr = reach_tools.ReachFeatureLayer.from_url(gis, reach_centroid_url)
reach_centroid_lyr

<ReachFeatureLayer url:"https://services3.arcgis.com/p9emohPzcQpdUrEc/arcgis/rest/services/Reach_Centroid/FeatureServer">

In [34]:
reach_fs = reach_centroid_lyr.query_by_reach_id(reach.reach_id)
reach_fs

KeyError: 'count'

In [105]:
%%time

self = reach

endTime = datetime.utcnow()
startTime = endTime - timedelta(hours=6)

endTimeStamp = round(endTime.timestamp())
startTimeStamp = round(startTime.timestamp())

url = f'http://www.americanwhitewater.org/api/gauge/{reach.gauge_id}/flows/{reach.gauge_metric}'

params = {
    'from': startTimeStamp,
    'to': endTimeStamp,
    'resolution': 1
}

resp = requests.get(url, params=params)

df = pd.DataFrame.from_dict(resp.json())
df.updated = df.updated.apply(lambda val: datetime.fromtimestamp(round(float(val))))

CPU times: user 28.4 ms, sys: 4.62 ms, total: 33.1 ms
Wall time: 479 ms


In [95]:
resp_json = resp.json()

for obs in resp_json:
    obs['updated'] = datetime.fromtimestamp(round(float(obs['updated'])))
resp_json

[{'gauge_id': '35665',
  'metric': '2',
  'nv': '1555025280',
  'reading': '2550',
  'updated': datetime.datetime(2019, 4, 11, 16, 28),
  'data': None,
  'id': '153180202990015'},
 {'gauge_id': '35665',
  'metric': '2',
  'nv': '1555028992',
  'reading': '2430',
  'updated': datetime.datetime(2019, 4, 11, 17, 29, 52),
  'data': None,
  'id': '153180202990465'}]

In [90]:
reach.gauge_min

1100.0

In [91]:
reach.gauge_max

1500.0